# Tools

## 1.- Setup inicial

### 1.1- Instalar librerías 

In [4]:
# Instalar librerías
#!pip install openai
#!pip install tenacity
#!pip install python-dotenv
#!spacy download es_core_news_sm

### 1.2.- Cargar librerías

In [5]:
import os
import json
import openai
import datetime
from dotenv import load_dotenv
from typing import Callable, List
from tenacity import retry, wait_random_exponential, stop_after_attempt

### 1.3.- OpenAI API

In [6]:
# Cargar secretos y configuración desde el archivo .env
load_dotenv()

# Configurar la clave de la API de OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
print("OpenAI API key: {}".format(openai.api_key[:5] + '...' + openai.api_key[-5:]))

# Nombres de los modelos
gpt35_model = os.getenv("OPENAI_GPT35_MODEL")
gpt35_16k_model = os.getenv("OPENAI_GPT35_16K_MODEL")
gpt4_model = os.getenv("OPENAI_GPT4_MODEL")
print("GPT-3.5-Turbo model: {}".format(gpt35_model))
print("GPT-3.5-Turbo-16k model: {}".format(gpt35_16k_model))
print("GPT-4 model: {}".format(gpt4_model))

OpenAI API key: sk-eO...Re5lw
GPT-3.5-Turbo model: gpt-3.5-turbo
GPT-3.5-Turbo-16k model: gpt-3.5-turbo-16k
GPT-4 model: gpt-4


### 1.4.- Importar clase Tools

In [7]:
from agents.tools import Tool, Parameter

### 1.5.- Definir funciones útiles

In [8]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(3))
def generate_text(prompt, model=gpt35_model, messages=[], max_tokens=200, temperature=1.0, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, stop=None):
    _messages = []
    _messages.extend(messages)
    _messages.append({"role": "user", "content": prompt})

    response = openai.ChatCompletion.create(
        model=model,
        messages=_messages,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        stop=stop
    )
    return response["choices"][0]["message"]["content"]

In [9]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(3))
def generate_text_with_function_call(prompt, model=gpt35_16k_model, messages=[], functions=[], max_tokens=-1, temperature=1.0, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, stop=None, function_call='auto'):
    _messages = []
    _messages.extend(messages)
    _messages.append({"role": "user", "content": prompt})
    
    if max_tokens == -1:
        max_tokens = None

    response = openai.ChatCompletion.create(
        model=model,
        messages=_messages,
        functions=functions,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        stop=stop,
        function_call=function_call
    )
    
    if "function_call" in response["choices"][0]["message"]:
        return response["choices"][0]["message"]["function_call"]
    else:
        return None

## 2.- Ejemplo Function Calling

### 2.1 Suma de números o concatenación de strings

#### 2.1.1 Definición de funciones

In [10]:
def sumar(a: int, b: int) -> int:
    return a + b

def concatenar_strings(s1: str, s2: str) -> str:
    return s1 + s2

#### 2.1.2 Instanciar funciones como Tools

In [11]:
sumar_tool = Tool(
    name="sumar",
    func=sumar,
    description="Esta función recibe dos números enteros como entrada y devuelve la suma de estos dos números.",
    arguments=[
        Parameter(name="Sumando 1", description="Primer número", type=int, required=True),
        Parameter(name="Sumando 2", description="Segundo número", type=int, required=True)
    ]
)

concatenar_tool = Tool(
    name="concatenar_strings",
    func=concatenar_strings,
    description="Esta función toma dos palabras como entrada y las combina en una sola cadena de texto, sin agregar espacios ni caracteres adicionales.",
    arguments=[
        Parameter(name="String 1", description="Primera cadena", type=str, required=True),
        Parameter(name="String 2", description="Segunda cadena", type=str, required=True)
    ]
)

#Definir lista de tools
tools1 = [sumar_tool, concatenar_tool]

#### 2.1.3 Ejemplos de uso

#### Sumar números

In [12]:
input1 = "Sumar los números 5 y 7"

##### Llamada al modelo

In [13]:
function_calling1 = generate_text_with_function_call(prompt = input1, model = gpt35_16k_model, functions = [sumar_tool.get_schema(), concatenar_tool.get_schema()])
print(function_calling1)

{
  "name": "sumar",
  "arguments": "{\n  \"Sumando 1\": 5,\n  \"Sumando 2\": 7\n}"
}


##### Se ejecuta la función

In [14]:
name1, arguments1 = function_calling1.values()
sum1, sum2 = json.loads(arguments1).values()
for tool in tools1:
    if name1 == tool.name:
        result = tool.execute(sum1, sum2)
        print(f'El resultado es: {result}')

El resultado es: 12


---------------------------------------------------------------------------------------------------------------------------------------------------------

#### Sumar strings

In [15]:
input2 = 'Une estas palabras: "hola" y "mundo"'

##### Llamada al modelo

In [16]:
function_calling2 = generate_text_with_function_call(prompt = input2, model = gpt35_16k_model, functions = [sumar_tool.get_schema(), concatenar_tool.get_schema()])
print(function_calling2)

{
  "name": "concatenar_strings",
  "arguments": "{\n  \"String 1\": \"hola\",\n  \"String 2\": \"mundo\"\n}"
}


##### Se ejecuta la función

In [17]:
name2, arguments2 = function_calling2.values()
arg1, arg2 = json.loads(arguments2).values()
for tool in tools1:
    if name2 == tool.name:
        result = tool.execute(arg1, arg2)
        print(f'El resultado es: {result}')

El resultado es: holamundo


## 3.- Ejemplo de Prompting para replicar Function Calling

### 3.1 Operaciones con números

#### 3.1.1 Definición de funciones

In [18]:
# Funciones
def sumar(a: int, b: int) -> int:
    return a + b

def multiplicar(a: int, b: int) -> int:
    return a * b

def restar(a: int, b: int) -> int:
    return a - b

def al_cuadrado(numero: int) -> int:
    return numero * numero

#### 3.1.2 Instanciar funciones como Tools

In [19]:
# Crear instancias de la clase Tool para cada función
sumar_tool = Tool(
    name="sumar",
    func=sumar,
    description="Sumar dos números",
    arguments=[
        Parameter(name="a", description="Primer número", type=int, required=True),
        Parameter(name="b", description="Segundo número", type=int, required=True)
    ]
)

multiplicar_tool = Tool(
    name="multiplicar",
    func=multiplicar,
    description="Multiplicar dos números",
    arguments=[
        Parameter(name="a", description="Primer número", type=int, required=True),
        Parameter(name="b", description="Segundo número", type=int, required=True)
    ]
)

restar_tool = Tool(
    name="restar",
    func=restar,
    description="Restar dos números",
    arguments=[
        Parameter(name="a", description="Primer número", type=int, required=True),
        Parameter(name="b", description="Segundo número", type=int, required=True)
    ]
)

cuadrado_tool = Tool(
    name="al_cuadrado",
    func=al_cuadrado,
    description="Calcular el cuadrado de un número",
    arguments=[
        Parameter(name="numero", description="Número para elevar al cuadrado", type=int, required=True)
    ]
)

# Definir lista de herramientas
tools2 = [sumar_tool, multiplicar_tool, restar_tool, cuadrado_tool]

#### 3.1.3 Técnicas de prompting

#### Prompt

In [20]:
prompt_principal = "Suma 4 y 2, luego el resultado multiplícalo por 2"

#### Esquema de funciones

In [21]:
functions = [sumar_tool.get_schema(), multiplicar_tool.get_schema(), restar_tool.get_schema(), cuadrado_tool.get_schema()]

#### Zero Shot

In [22]:
system_message = {
    "role": "system", "content": "Considera la prompt. Evalúa las acciones que se describen y según eso y basado en la descripción de las <FUNCIONES> que te adjuntaré, determina qué función es necesario utilizar y extrae los parámetros necesarios. Retorna solo la función y los parámetros.",
    "role": "system", "content": f"<FUNCIONES> {functions}",
}
print(generate_text(prompt_principal, messages=[system_message], model=gpt35_16k_model))

Para realizar la operación que mencionaste, podemos utilizar las funciones "sumar" y "multiplicar" que tenemos disponibles.

Primero, vamos a sumar 4 y 2 utilizando la función "sumar":

```
sumar(4, 2)
```

Esto nos dará como resultado 6.

Luego, vamos a multiplicar el resultado obtenido por 2 utilizando la función "multiplicar":

```
multiplicar(6, 2)
```

Esto nos dará como resultado 12.

Por lo tanto, la respuesta es 12.


#### Few Shot

In [23]:
system_message = { "role": "system", "content": f"<FUNCIONES> {functions}"}
prompt = f"""3 veces 5, más 4. // función: "multiplicar", a:3, b:5. Luego, función: "sumar", a: (resultado (3 * 5)), b: 4. Resultado: 19  
Tres al cuadrado, menos 6. // función: "al_cuadrado", numero:3. Luego, función: "restar", a: (resultado (3 * 3)), b: 6. Resultado: 3
Seis más cinco, menos 7 // función: "sumar", a: 6, b: 5. Luego, función: "restar", a: (resultado (6+5)), b: 7. Resultado: 4
{prompt_principal}. // """
print(generate_text(prompt, messages=[system_message], model=gpt35_16k_model))

función: "sumar", a: 4, b: 2. Luego, función: "multiplicar", a: (resultado (4 + 2)), b: 2. Resultado: 12


#### Chain of Thought

In [32]:
system_message = { "role": "system", "content": f"<FUNCIONES> {functions}"}

prompt = f"""P: Ocho menos dos al cuadrado.
Pregunta Intermedia: Hay alguna o más de una palabra en la prompt que haga alusión a alguna de las <FUNCIONES>?
Respuesta Intermedia: Sí, al cuadrado está directamente relacionada con una de las funciones. Y 'menos' se puede relacionar con la función "restar".
Pregunta Intermedia: Hay alguna otra palabra o acción relacionada con las <FUNCIONES>?
Respuesta Intermedia: No
Pregunta Intermedia: De acuerdo al orden de prioridad de las operaciones matemáticas, (1) Paréntesis > (2) Potencias > (3) Multiplicación > (4) División > (5) Adición (Suma) > (6) Sustracción (Resta), cuál operación se realiza primero?
Respuesta Intermedia: La Potencia ("al cuadrado") tiene mayor prioridad que la Resta ("restar")
Pregunta Intermedia: Qué número está asociado a la operación de "al_cuadrado"
Respuesta Intermedia: "dos" es equivalente a 2 y está justo antes de "al cuadrado". Entonces, Función: "al_cuadrado", numero: 2.
Pregunta Intermedia: Qué función corresponde ahora?
Respuesta Intermedia: "restar".
Pregunta Intermedia: Cuántos argumentos recibe la función "restar"?
Respuesta Intermedia: a: int, b: int
Pregunta Intermedia: En la prompt (P) podemos identificar dos int, cuáles?
Respuesta Intermedia: Sí, ocho: equivalente a 8, y (dos al cuadrado), que resolvimos antes
Pregunta Intermedia: Entonces, qué función hay que realizar ahora y cuáles son los argumentos?
Respuesta Intermedia: Función: "restar", a: 8, b: (2 al cuadrado)
Respuesta Final: Primero la función: "al cuadrado", numero: 2 -> Resultado: 4. Luego, "restar", a: 8, b: 4 -> Resultado final: 4

P: {prompt_principal}
"""
print(generate_text(prompt, messages = [system_message], max_tokens=500))

Pregunta Intermedia: Hay alguna o más de una palabra en la prompt que haga alusión a alguna de las <FUNCIONES>?
Respuesta Intermedia: Sí, "suma" se relaciona con la función "sumar" y "multiplica" se relaciona con la función "multiplicar".
Pregunta Intermedia: Hay alguna otra palabra o acción relacionada con las <FUNCIONES>?
Respuesta Intermedia: Sí, "multiplícalo" se relaciona con la función "multiplicar".
Pregunta Intermedia: De acuerdo al orden de prioridad de las operaciones matemáticas, (1) Paréntesis > (2) Potencias > (3) Multiplicación > (4) División > (5) Adición (Suma) > (6) Sustracción (Resta), cuál operación se realiza primero?
Respuesta Intermedia: La Multiplicación tiene mayor prioridad que la Suma.
Pregunta Intermedia: Cuántos argumentos recibe la función "sumar"?
Respuesta Intermedia: a: int, b: int
Pregunta Intermedia: En la prompt (P) hay dos int, cuáles son?
Respuesta Intermedia: Sí, 4 y 2 son valores int.
Pregunta Intermedia: Entonces, qué función hay que realizar aho